# 🏠📊 Regresión Lineal Múltiple - Predicción de Precios de Viviendas

---

## 📋 Índice de Contenidos

1. [🎯 **Introducción y Objetivos**](#introduccion)
2. [📦 **Importación de Librerías**](#librerias)
3. [📊 **Carga y Exploración de Datos**](#datos)
4. [🔍 **Análisis Exploratorio de Datos (EDA)**](#eda)
5. [🏗️ **Construcción del Modelo**](#modelo)
   - 5.1 [📐 Definición de Variables](#variables)
   - 5.2 [⚙️ Creación y Ajuste del Modelo](#ajuste)
   - 5.3 [📈 Resultados del Modelo](#resultados)
6. [🔬 **Interpretación de Resultados**](#interpretacion)
7. [📊 **Visualizaciones**](#visualizaciones)
8. [✅ **Validación del Modelo**](#validacion)
9. [🎯 **Predicciones**](#predicciones)
10. [📝 **Conclusiones y Recomendaciones**](#conclusiones)

---

## 🎯 1. Introducción y Objetivos

### 🏠 ¿Qué es la Regresión Lineal Múltiple?

La **regresión lineal múltiple** es una técnica estadística que nos permite predecir el valor de una variable dependiente (en nuestro caso, el precio de las viviendas) basándose en múltiples variables independientes (características de las casas).

### 🎯 Objetivos del Análisis:

- 🔍 **Identificar** qué factores influyen más en el precio de las viviendas
- 📈 **Construir** un modelo predictivo confiable
- 💡 **Interpretar** los coeficientes del modelo
- 🎯 **Realizar** predicciones precisas de precios

### 📊 Dataset: Boston Housing (Sintético)

Trabajaremos con un dataset sintético basado en el famoso Boston Housing dataset, que contiene información sobre precios de viviendas y sus características.

---

## 📦 2. Importación de Librerías

Importamos las librerías necesarias para nuestro análisis:

In [ ]:
# 📊 Manipulación y análisis de datos
import pandas as pd
import numpy as np

# 📈 Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# 🔬 Modelado estadístico
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# ⚙️ Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# 🚫 Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas exitosamente!")

## 📊 3. Carga y Exploración de Datos

### 📁 Cargando el Dataset

In [ ]:
# 📁 Cargar el dataset
df_boston = pd.read_csv("boston_sintetico.csv")

print(f"📊 Dataset cargado exitosamente!")
print(f"📏 Dimensiones: {df_boston.shape[0]} filas × {df_boston.shape[1]} columnas")
print("\n🔍 Primeras 5 filas del dataset:")
df_boston.head()

## 🔍 4. Análisis Exploratorio de Datos (EDA)

### 📋 Información General del Dataset

In [ ]:
# 📊 Información general
print("📊 INFORMACIÓN GENERAL DEL DATASET")
print("=" * 50)
print(f"📏 Filas: {df_boston.shape[0]}")
print(f"📐 Columnas: {df_boston.shape[1]}")
print(f"💾 Memoria utilizada: {df_boston.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n🏷️ TIPOS DE DATOS:")
print(df_boston.dtypes)

print("\n❓ VALORES FALTANTES:")
missing_values = df_boston.isnull().sum()
if missing_values.sum() == 0:
    print("✅ ¡Excelente! No hay valores faltantes en el dataset")
else:
    print(missing_values[missing_values > 0])

In [ ]:
# 📊 Descripción de variables
print("📊 DESCRIPCIÓN DE LAS VARIABLES")
print("=" * 50)

variables_info = {
    "🏠 Rooms": "Número promedio de habitaciones por vivienda",
    "📍 Distance_to_Center": "Distancia al centro de la ciudad (km)",
    "🕰️ House_Age": "Edad de la vivienda (años)",
    "🚨 Crime_Rate": "Tasa de criminalidad per cápita",
    "👨‍🏫 Student_Teacher_Ratio": "Ratio estudiante-profesor en escuelas locales",
    "🏘️ Neighborhood": "Tipo de vecindario",
    "🏗️ House_Type": "Tipo de vivienda",
    "💰 House_Price": "Precio de la vivienda (miles de USD) - VARIABLE OBJETIVO"
}

for var, desc in variables_info.items():
    print(f"{var}: {desc}")

In [ ]:
# 📊 Estadísticas descriptivas
print("📊 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 50)

# Variables numéricas
numeric_cols = df_boston.select_dtypes(include=[np.number]).columns
print("\n🔢 VARIABLES NUMÉRICAS:")
df_boston[numeric_cols].describe().round(2)

In [ ]:
# 📊 Variables categóricas
print("📊 ANÁLISIS DE VARIABLES CATEGÓRICAS")
print("=" * 50)

categorical_cols = df_boston.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n🏷️ {col.upper()}:")
    value_counts = df_boston[col].value_counts()
    percentages = (value_counts / len(df_boston) * 100).round(1)
    
    for value, count in value_counts.items():
        percentage = percentages[value]
        print(f"   • {value}: {count} ({percentage}%)")

## 🏗️ 5. Construcción del Modelo de Regresión Lineal Múltiple

### 📐 5.1 Definición de Variables

Para nuestro modelo inicial, utilizaremos las variables más relevantes:

In [ ]:
print("📐 DEFINICIÓN DE VARIABLES PARA EL MODELO")
print("=" * 50)

# 🎯 Variable dependiente (lo que queremos predecir)
y = df_boston["House_Price"]
print(f"🎯 Variable Dependiente (Y): House_Price")
print(f"   📊 Rango: ${y.min():.2f}K - ${y.max():.2f}K")
print(f"   📈 Promedio: ${y.mean():.2f}K")

# 📊 Variables independientes (predictores)
X = df_boston[["Rooms", "Distance_to_Center"]]
print(f"\n📊 Variables Independientes (X):")
for col in X.columns:
    print(f"   • {col}")

print(f"\n✅ Variables definidas correctamente!")
print(f"📏 Dimensiones de X: {X.shape}")
print(f"📏 Dimensiones de y: {y.shape}")

### ⚙️ 5.2 Creación y Ajuste del Modelo

Ahora crearemos nuestro modelo de regresión lineal múltiple:

In [ ]:
print("⚙️ CREACIÓN DEL MODELO DE REGRESIÓN LINEAL MÚLTIPLE")
print("=" * 60)

# 🔧 Agregar constante al modelo (intercepto)
X_with_const = sm.add_constant(X)
print("✅ Constante agregada al modelo")

# 🏗️ Crear y ajustar el modelo
modelo_ols = sm.OLS(y, X_with_const).fit()
print("✅ Modelo ajustado exitosamente")

print(f"\n📊 El modelo ha sido entrenado con {len(y)} observaciones")
print(f"🎯 Fórmula del modelo: House_Price = β₀ + β₁(Rooms) + β₂(Distance_to_Center) + ε")

### 📈 5.3 Resultados del Modelo

Veamos los resultados detallados de nuestro modelo:

In [ ]:
print("📈 RESULTADOS COMPLETOS DEL MODELO")
print("=" * 50)

# 📊 Mostrar resumen completo
modelo_ols.summary()

## 🔬 6. Interpretación de Resultados

### 📊 Análisis de Coeficientes

In [ ]:
print("🔬 INTERPRETACIÓN DETALLADA DE LOS COEFICIENTES")
print("=" * 60)

# 📊 Extraer coeficientes
coeficientes = modelo_ols.params
p_values = modelo_ols.pvalues
conf_int = modelo_ols.conf_int()

print("\n📈 ECUACIÓN DEL MODELO:")
print(f"House_Price = {coeficientes['const']:.4f} + {coeficientes['Rooms']:.4f}(Rooms) + {coeficientes['Distance_to_Center']:.4f}(Distance_to_Center)")

print("\n🔍 INTERPRETACIÓN DE CADA COEFICIENTE:")
print("-" * 50)

# Intercepto
print(f"🏠 INTERCEPTO (β₀): {coeficientes['const']:.4f}")
print(f"   💡 Interpretación: Precio base cuando Rooms=0 y Distance_to_Center=0")
print(f"   📊 P-value: {p_values['const']:.6f}")
print(f"   ✅ Significativo: {'Sí' if p_values['const'] < 0.05 else 'No'}")

# Rooms
print(f"\n🏠 ROOMS (β₁): {coeficientes['Rooms']:.4f}")
print(f"   💡 Interpretación: Por cada habitación adicional, el precio aumenta ${coeficientes['Rooms']:.2f}K")
print(f"   📊 P-value: {p_values['Rooms']:.6f}")
print(f"   ✅ Significativo: {'Sí' if p_values['Rooms'] < 0.05 else 'No'}")

# Distance_to_Center
print(f"\n📍 DISTANCE_TO_CENTER (β₂): {coeficientes['Distance_to_Center']:.4f}")
print(f"   💡 Interpretación: Por cada km adicional del centro, el precio {'disminuye' if coeficientes['Distance_to_Center'] < 0 else 'aumenta'} ${abs(coeficientes['Distance_to_Center']):.2f}K")
print(f"   📊 P-value: {p_values['Distance_to_Center']:.6f}")
print(f"   ✅ Significativo: {'Sí' if p_values['Distance_to_Center'] < 0.05 else 'No'}")

In [ ]:
print("📊 MÉTRICAS DE CALIDAD DEL MODELO")
print("=" * 50)

# 📈 R-cuadrado
r2 = modelo_ols.rsquared
r2_adj = modelo_ols.rsquared_adj

print(f"📈 R² (Coeficiente de Determinación): {r2:.4f}")
print(f"   💡 El modelo explica el {r2*100:.2f}% de la variabilidad en los precios")

print(f"\n📈 R² Ajustado: {r2_adj:.4f}")
print(f"   💡 Ajustado por el número de variables: {r2_adj*100:.2f}%")

# 📊 F-statistic
f_stat = modelo_ols.fvalue
f_pvalue = modelo_ols.f_pvalue

print(f"\n📊 F-Statistic: {f_stat:.2f}")
print(f"📊 P-value (F-test): {f_pvalue:.2e}")
print(f"✅ Modelo globalmente significativo: {'Sí' if f_pvalue < 0.05 else 'No'}")

# 📏 AIC y BIC
aic = modelo_ols.aic
bic = modelo_ols.bic

print(f"\n📏 AIC (Criterio de Información de Akaike): {aic:.2f}")
print(f"📏 BIC (Criterio de Información Bayesiano): {bic:.2f}")
print(f"   💡 Valores más bajos indican mejor ajuste")

## 📊 7. Visualizaciones del Modelo

### 📈 Gráficos de Dispersión y Correlaciones

In [ ]:
# 📊 Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = df_boston[['Rooms', 'Distance_to_Center', 'House_Age', 'Crime_Rate', 'Student_Teacher_Ratio', 'House_Price']].corr()

sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True,
            fmt='.3f',
            cbar_kws={'label': 'Correlación'})

plt.title('🔥 Matriz de Correlación - Variables Numéricas', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("💡 Interpretación de la Matriz de Correlación:")
print("   • Valores cercanos a +1: Correlación positiva fuerte")
print("   • Valores cercanos a -1: Correlación negativa fuerte")
print("   • Valores cercanos a 0: Sin correlación lineal")

In [ ]:
# 📊 Gráficos de dispersión
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Rooms vs House_Price
axes[0].scatter(df_boston['Rooms'], df_boston['House_Price'], alpha=0.6, color='skyblue', edgecolors='navy')
axes[0].set_xlabel('🏠 Número de Habitaciones', fontsize=12)
axes[0].set_ylabel('💰 Precio de la Casa (K USD)', fontsize=12)
axes[0].set_title('🏠 Habitaciones vs Precio', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Distance_to_Center vs House_Price
axes[1].scatter(df_boston['Distance_to_Center'], df_boston['House_Price'], alpha=0.6, color='lightcoral', edgecolors='darkred')
axes[1].set_xlabel('📍 Distancia al Centro (km)', fontsize=12)
axes[1].set_ylabel('💰 Precio de la Casa (K USD)', fontsize=12)
axes[1].set_title('📍 Distancia vs Precio', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 📊 Análisis de residuos
residuos = modelo_ols.resid
valores_ajustados = modelo_ols.fittedvalues

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Residuos vs Valores Ajustados
axes[0,0].scatter(valores_ajustados, residuos, alpha=0.6, color='purple')
axes[0,0].axhline(y=0, color='red', linestyle='--', alpha=0.8)
axes[0,0].set_xlabel('Valores Ajustados', fontsize=12)
axes[0,0].set_ylabel('Residuos', fontsize=12)
axes[0,0].set_title('📊 Residuos vs Valores Ajustados', fontsize=14, fontweight='bold')
axes[0,0].grid(True, alpha=0.3)

# 2. Q-Q Plot de residuos
sm.qqplot(residuos, line='s', ax=axes[0,1])
axes[0,1].set_title('📈 Q-Q Plot de Residuos', fontsize=14, fontweight='bold')
axes[0,1].grid(True, alpha=0.3)

# 3. Histograma de residuos
axes[1,0].hist(residuos, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1,0].set_xlabel('Residuos', fontsize=12)
axes[1,0].set_ylabel('Frecuencia', fontsize=12)
axes[1,0].set_title('📊 Distribución de Residuos', fontsize=14, fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

# 4. Residuos vs Orden
axes[1,1].plot(residuos, 'o-', alpha=0.6, color='orange')
axes[1,1].axhline(y=0, color='red', linestyle='--', alpha=0.8)
axes[1,1].set_xlabel('Orden de Observación', fontsize=12)
axes[1,1].set_ylabel('Residuos', fontsize=12)
axes[1,1].set_title('📈 Residuos vs Orden', fontsize=14, fontweight='bold')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔍 INTERPRETACIÓN DE LOS GRÁFICOS DE RESIDUOS:")
print("   📊 Residuos vs Valores Ajustados: Debe mostrar patrón aleatorio")
print("   📈 Q-Q Plot: Los puntos deben seguir la línea diagonal")
print("   📊 Histograma: Los residuos deben seguir distribución normal")
print("   📈 Residuos vs Orden: No debe haber patrones temporales")

## ✅ 8. Validación del Modelo

### 🔄 División Train-Test

In [ ]:
print("✅ VALIDACIÓN DEL MODELO")
print("=" * 50)

# 🔄 División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"📊 Datos de entrenamiento: {X_train.shape[0]} observaciones")
print(f"📊 Datos de prueba: {X_test.shape[0]} observaciones")

# 🏗️ Entrenar modelo con sklearn para comparación
modelo_sklearn = LinearRegression()
modelo_sklearn.fit(X_train, y_train)

# 🎯 Predicciones
y_pred_train = modelo_sklearn.predict(X_train)
y_pred_test = modelo_sklearn.predict(X_test)

# 📊 Métricas de evaluación
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

print("\n📈 MÉTRICAS DE RENDIMIENTO:")
print("-" * 40)
print(f"📊 R² Entrenamiento: {r2_train:.4f}")
print(f"📊 R² Prueba: {r2_test:.4f}")
print(f"📊 RMSE Entrenamiento: ${rmse_train:.2f}K")
print(f"📊 RMSE Prueba: ${rmse_test:.2f}K")

# 🔍 Evaluación de sobreajuste
diferencia_r2 = abs(r2_train - r2_test)
print(f"\n🔍 EVALUACIÓN DE SOBREAJUSTE:")
print(f"📊 Diferencia R²: {diferencia_r2:.4f}")
if diferencia_r2 < 0.05:
    print("✅ Modelo bien balanceado (sin sobreajuste significativo)")
elif diferencia_r2 < 0.1:
    print("⚠️ Posible sobreajuste leve")
else:
    print("❌ Sobreajuste significativo")

In [ ]:
# 📊 Gráfico de valores reales vs predichos
plt.figure(figsize=(12, 5))

# Entrenamiento
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_pred_train, alpha=0.6, color='blue', label='Datos')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2, label='Línea perfecta')
plt.xlabel('💰 Precio Real (K USD)', fontsize=12)
plt.ylabel('💰 Precio Predicho (K USD)', fontsize=12)
plt.title(f'🏋️ Entrenamiento\nR² = {r2_train:.3f}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Prueba
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_test, alpha=0.6, color='green', label='Datos')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Línea perfecta')
plt.xlabel('💰 Precio Real (K USD)', fontsize=12)
plt.ylabel('💰 Precio Predicho (K USD)', fontsize=12)
plt.title(f'🧪 Prueba\nR² = {r2_test:.3f}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 9. Realizando Predicciones

### 🏠 Ejemplos de Predicción

In [ ]:
print("🎯 EJEMPLOS DE PREDICCIÓN")
print("=" * 50)

# 🏠 Ejemplos de casas para predecir
ejemplos_casas = [
    {"Rooms": 6.0, "Distance_to_Center": 1.0, "Descripción": "🏠 Casa promedio cerca del centro"},
    {"Rooms": 8.0, "Distance_to_Center": 0.5, "Descripción": "🏰 Casa grande muy cerca del centro"},
    {"Rooms": 4.0, "Distance_to_Center": 3.0, "Descripción": "🏡 Casa pequeña en las afueras"},
    {"Rooms": 7.0, "Distance_to_Center": 2.0, "Descripción": "🏠 Casa grande en zona residencial"}
]

print("\n🔮 PREDICCIONES:")
print("-" * 60)

for i, casa in enumerate(ejemplos_casas, 1):
    # Crear DataFrame para la predicción
    nueva_casa = pd.DataFrame({
        'Rooms': [casa['Rooms']],
        'Distance_to_Center': [casa['Distance_to_Center']]
    })
    
    # Predicción
    precio_predicho = modelo_sklearn.predict(nueva_casa)[0]
    
    print(f"\n{i}. {casa['Descripción']}")
    print(f"   📊 Habitaciones: {casa['Rooms']}")
    print(f"   📍 Distancia al centro: {casa['Distance_to_Center']} km")
    print(f"   💰 Precio predicho: ${precio_predicho:.2f}K USD")
    
    # Calcular usando la ecuación manual
    precio_manual = (coeficientes['const'] + 
                    coeficientes['Rooms'] * casa['Rooms'] + 
                    coeficientes['Distance_to_Center'] * casa['Distance_to_Center'])
    print(f"   🧮 Verificación manual: ${precio_manual:.2f}K USD")

In [ ]:
# 🎮 Función para predicción interactiva
def predecir_precio_casa(habitaciones, distancia_centro):
    """
    🏠 Función para predecir el precio de una casa
    
    Parámetros:
    - habitaciones: Número de habitaciones
    - distancia_centro: Distancia al centro en km
    
    Retorna:
    - Precio predicho en miles de USD
    """
    nueva_casa = pd.DataFrame({
        'Rooms': [habitaciones],
        'Distance_to_Center': [distancia_centro]
    })
    
    precio = modelo_sklearn.predict(nueva_casa)[0]
    
    print(f"🏠 PREDICCIÓN DE PRECIO")
    print(f"📊 Habitaciones: {habitaciones}")
    print(f"📍 Distancia al centro: {distancia_centro} km")
    print(f"💰 Precio estimado: ${precio:.2f}K USD")
    print(f"💵 Precio estimado: ${precio*1000:,.0f} USD")
    
    return precio

# 🎯 Ejemplo de uso
print("🎮 PREDICCIÓN INTERACTIVA")
print("=" * 50)
precio_ejemplo = predecir_precio_casa(6.5, 1.5)

print("\n💡 ¡Puedes usar esta función para predecir el precio de cualquier casa!")

## 📝 10. Conclusiones y Recomendaciones

### 🎯 Resumen de Hallazgos

In [ ]:
print("📝 RESUMEN EJECUTIVO DEL ANÁLISIS")
print("=" * 60)

print("\n🎯 HALLAZGOS PRINCIPALES:")
print("-" * 40)

print(f"\n1. 📊 PODER PREDICTIVO DEL MODELO:")
print(f"   • R² = {r2:.3f} ({r2*100:.1f}% de variabilidad explicada)")
print(f"   • RMSE = ${rmse_test:.2f}K USD (error promedio)")
print(f"   • Modelo {'BUENO' if r2 > 0.4 else 'REGULAR' if r2 > 0.2 else 'POBRE'}")

print(f"\n2. 🏠 IMPACTO DE LAS HABITACIONES:")
print(f"   • Coeficiente: +${coeficientes['Rooms']:.2f}K por habitación")
print(f"   • Cada habitación adicional aumenta el precio en ${coeficientes['Rooms']:.2f}K USD")
print(f"   • Variable {'MUY SIGNIFICATIVA' if p_values['Rooms'] < 0.001 else 'SIGNIFICATIVA'}")

print(f"\n3. 📍 IMPACTO DE LA DISTANCIA:")
print(f"   • Coeficiente: ${coeficientes['Distance_to_Center']:.2f}K por km")
print(f"   • Cada km adicional del centro reduce el precio en ${abs(coeficientes['Distance_to_Center']):.2f}K USD")
print(f"   • Variable {'MUY SIGNIFICATIVA' if p_values['Distance_to_Center'] < 0.001 else 'SIGNIFICATIVA'}")

print(f"\n4. ✅ VALIDACIÓN DEL MODELO:")
print(f"   • R² Entrenamiento: {r2_train:.3f}")
print(f"   • R² Prueba: {r2_test:.3f}")
print(f"   • Diferencia: {abs(r2_train - r2_test):.3f}")
print(f"   • Estado: {'SIN SOBREAJUSTE' if abs(r2_train - r2_test) < 0.05 else 'POSIBLE SOBREAJUSTE'}")

### 💡 Recomendaciones para Mejorar el Modelo

#### 🔧 Mejoras Técnicas:

1. **📊 Incluir más variables predictoras:**
   - `House_Age` (edad de la casa)
   - `Crime_Rate` (tasa de criminalidad)
   - `Student_Teacher_Ratio` (calidad educativa)
   - Variables categóricas como `Neighborhood` y `House_Type`

2. **🔄 Transformaciones de variables:**
   - Aplicar transformaciones logarítmicas si es necesario
   - Crear variables de interacción
   - Normalizar variables si tienen escalas muy diferentes

3. **🧪 Validación más robusta:**
   - Implementar validación cruzada k-fold
   - Probar diferentes divisiones train-test
   - Analizar residuos más detalladamente

#### 🎯 Aplicaciones Prácticas:

- **🏘️ Tasación inmobiliaria:** Estimar precios de propiedades
- **💼 Inversión:** Identificar propiedades subvaloradas
- **🏗️ Desarrollo urbano:** Planificar nuevos proyectos
- **📊 Análisis de mercado:** Entender factores que afectan precios

---

### 🎉 ¡Análisis Completado!

Has completado exitosamente un análisis completo de regresión lineal múltiple. Este modelo te permite:

- ✅ **Predecir** precios de viviendas basándose en características clave
- ✅ **Entender** qué factores más influyen en los precios
- ✅ **Cuantificar** el impacto de cada variable
- ✅ **Validar** la calidad del modelo

**🚀 ¡Continúa explorando y mejorando tu modelo!**

---

*📚 Para más información sobre regresión lineal múltiple, consulta recursos adicionales de estadística y machine learning.*